In [10]:
import pandas as pd

In [11]:
import os

## Ввести абсолютный путь к файлам

In [63]:
# path = "C:\\Users\\mi\\jupyterProjects\\tom's work"
path = "/Users/valeriali/JupyterProjects/tom's work"

In [64]:
os.chdir(path)

In [65]:
os.getcwd()

"/Users/valeriali/JupyterProjects/tom's work"

In [66]:
#настройки для отображения всех колонок
pd.set_option('display.max_columns', None)

### Считаваем файлы req-implements

In [67]:
for i in os.listdir("req_implement"):
    print(i)

.DS_Store
req-implement.txt


In [68]:
orders = pd.read_csv('req_implement/req-implement.txt', encoding='ISO-8859-5', sep='\t',header=None, parse_dates=[1], dayfirst=True)

In [69]:
columns_for_req = ['№ зкз.Покуп','Дат.зкз.Пок','КССС мат','Номенклатура материала', 'Заявлено Покупателем', 
                   'Не исполнено по продукту, тн', 'Исполнено опер.отгрузками', 'имя Покупателя', 'Вид транспортного средства', 
                   'СтКМ', 'Комментарий', 'ВремяСтКМ', 'ДатСтКМ', 'Время', 'ПланКОНдата', 'ПланНАЧдата', 'ИнкТм', 
                   'Район сбыта', 'Договор', 'ДатаСоздан', 'ДатаИзм', 'Канал сбыта', 'Пункт отгрузки', 'Предписания по таре', 
                   'Период', 'Инкот. 2', 'Регион ГП', 'Регион Грузополуч.', 'Маршрут', 'Пункт разгрузки', 'Тип дилера', 
                   'Исполнено факт.отгрузками', 'Фирменная / не фирменная проду', 'имя Грузополучателя', 'КССС Пок', 
                   'Условие отгрузки', 'СтКМ продукта', 'Стоимость', 'Валюта']
len(columns_for_req)

39

In [70]:
orders.columns = columns_for_req

In [71]:
orders.loc[(orders['Пункт отгрузки'] == 'ООО "ЛУКОЙЛ-Торжок"'), 'Пункт отгрузки'] = 'ООО "Тебойл Рус" Торжок'

In [72]:
orders['Пункт отгрузки'].unique()

array(['ООО "Тебойл Рус" Торжок', 'ООО "Тебойл Рус" Москва',
       'ООО "Тебойл Рус" С.-Петербург'], dtype=object)

In [73]:
orders.sort_values(by=['Дат.зкз.Пок','№ зкз.Покуп'], ascending=True, inplace=True)

In [74]:
orders['Пункт отгрузки'].unique()

array(['ООО "Тебойл Рус" Торжок', 'ООО "Тебойл Рус" Москва',
       'ООО "Тебойл Рус" С.-Петербург'], dtype=object)

In [75]:
orders['Заявлено Покупателем'] = orders['Заявлено Покупателем'].apply(lambda x: float(x.split()[0].replace(',', '.')))
orders['Не исполнено по продукту, тн'] = orders['Не исполнено по продукту, тн'].apply(lambda x: float(x.split()[0].replace(',', '.')))
orders['Исполнено опер.отгрузками'] = orders['Исполнено опер.отгрузками'].apply(lambda x: float(x.split()[0].replace(',', '.')))


In [76]:
orders

,№ зкз.Покуп,Дат.зкз.Пок,КССС мат,Номенклатура материала,Заявлено Покупателем,"Не исполнено по продукту, тн",Исполнено опер.отгрузками,имя Покупателя,Вид транспортного средства,СтКМ,Комментарий,ВремяСтКМ,ДатСтКМ,Время,ПланКОНдата,ПланНАЧдата,ИнкТм,Район сбыта,Договор,ДатаСоздан,ДатаИзм,Канал сбыта,Пункт отгрузки,Предписания по таре,Период,Инкот. 2,Регион ГП,Регион Грузополуч.,Маршрут,Пункт разгрузки,Тип дилера,Исполнено факт.отгрузками,Фирменная / не фирменная проду,имя Грузополучателя,КССС Пок,Условие отгрузки,СтКМ продукта,Стоимость,Валюта
778,5474799,2022-07-21,3426179,Масло Helix HX7 5W40 к.4л,17761.92,0.0,0.0,"ООО ""Вектор Авто""",Авто для фасовки,C,NaN,11:40:57,28.07.2022,15:04:34,31.07.2022,23.07.2022,CPT,Оптовые поставки,22L0730,21.07.2022,21.07.2022,Внутренний рынок,"ООО ""Тебойл Рус"" Торжок",средняя и мелкая фc.,202208,Москва,77,г. Москва,Автотранспорт (Торжок ->Москва),Г. МОСКВА 117574 ПР. ОДОЕВСКОГО Д. 2Б,NaN,"17761,920",NaN,"ООО ""Вектор Авто""",3485843,Автотранспорт,A,"9303054,26",RUB
938,5474824,2022-07-21,3426180,Масло Helix HX7 10W40 б.209л,18037.00,0.0,0.0,"ООО ""Вектор Авто""",Авто для фасовки,C,NaN,11:41:14,28.07.2022,15:53:26,31.07.2022,23.07.2022,CPT,Оптовые поставки,22L0730,21.07.2022,21.07.2022,Внутренний рынок,"ООО ""Тебойл Рус"" Торжок",крупная фасовка,202208,Москва,77,г. Москва,Автотранспорт (Торжок ->Москва),Г. МОСКВА 117574 ПР. ОДОЕВСКОГО Д. 2Б,NaN,"18037,000",NaN,"ООО ""Вектор Авто""",3485843,Автотранспорт,A,"6778593,19",RUB
51,5474946,2022-07-22,3451107,Масло Rimula R6 M 10W40 б.209л,9156.03,0.0,0.0,"ООО ""ГЛОБАЛ ТРАК СЕРВИС""",Авто для фасовки,C,NaN,16:54:05,27.07.2022,13:30:52,31.07.2022,24.07.2022,DDP,Оптовые поставки,19L0469,22.07.2022,22.07.2022,Внутренний рынок,"ООО ""Тебойл Рус"" Москва",крупная фасовка,202208,Москва,77,г. Москва,Автотранспорт (Балашиха ->Москва),"124460 МОСКВА, ЗЕЛЕНОГРАД, ПРОСПЕКТ ГЕНЕРАЛА ...",NaN,"9156,030",NaN,"ООО ""ГЛОБАЛ ТРАК СЕРВИС""",2094810,Автотранспорт,A,"5077091,88",RUB
52,5474946,2022-07-22,3451095,Масло Rimula R5 LE 10W30 б.209л,3607.40,0.0,0.0,"ООО ""ГЛОБАЛ ТРАК СЕРВИС""",Авто для фасовки,C,NaN,16:54:05,27.07.2022,13:30:52,31.07.2022,24.07.2022,DDP,Оптовые поставки,19L0469,22.07.2022,22.07.2022,Внутренний рынок,"ООО ""Тебойл Рус"" Москва",крупная фасовка,202208,Москва,77,г. Москва,Автотранспорт (Балашиха ->Москва),"124460 МОСКВА, ЗЕЛЕНОГРАД, ПРОСПЕКТ ГЕНЕРАЛА ...",NaN,"3607,400",NaN,"ООО ""ГЛОБАЛ ТРАК СЕРВИС""",2094810,Автотранспорт,A,"1442599,26",RUB
53,5474946,2022-07-22,3451135,Масло Spirax S3 G 80W б.209л,369.93,0.0,0.0,"ООО ""ГЛОБАЛ ТРАК СЕРВИС""",Авто для фасовки,C,NaN,16:54:05,27.07.2022,13:30:52,31.07.2022,24.07.2022,DDP,Оптовые поставки,19L0469,22.07.2022,22.07.2022,Внутренний рынок,"ООО ""Тебойл Рус"" Москва",крупная фасовка,202208,Москва,77,г. Москва,Автотранспорт (Балашиха ->Москва),"124460 МОСКВА, ЗЕЛЕНОГРАД, ПРОСПЕКТ ГЕНЕРАЛА ...",NaN,"369,930",NaN,"ООО ""ГЛОБАЛ ТРАК СЕРВИС""",2094810,Автотранспорт,A,"118867,39",RUB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,5485277,2022-08-30,3455335,Масло TERRION ENGINE R 10W40 б.175кг,4900.00,4900.0,0.0,"АО ""ГК ""ЛогЛаб""",Авто для фасовки,NaN,NaN,00:00:00,NaN,15:30:04,31.08.2022,31.08.2022,DDU,внутрикорп. Транзит,22L0227,30.08.2022,30.08.2022,Внутренний рынок,"ООО ""Тебойл Рус"" Торжок",крупная фасовка,202208,Москва,77,г. Москва,Автотранспорт (Торжок ->Москва),"МО, МЫТИЩИНСКИЙ Р-Н, С. П. ФЕДОСКИНСКОЕ, Д. Е...",NaN,"0,000",NaN,"АО ""ГК ""ЛогЛаб"" Мытищи",3422199,Автотранспорт,A,"1321236,00",RUB
47,5485277,2022-08-30,3455339,"Масло TERRION ENGINE R 10W40 к.21,5л",1632.00,1632.0,0.0,"АО ""ГК ""ЛогЛаб""",Авто для фасовки,NaN,NaN,00:00:00,NaN,15:30:04,31.08.2022,31.08.2022,DDU,внутрикорп. Транзит,22L0227,30.08.2022,30.08.2022,Внутренний рынок,"ООО ""Тебойл Рус"" Торжок",средняя и мелкая фc.,202208,Москва,77,г. Москва,Автотранспорт (Торжок ->Москва),"МО, МЫТИЩИНСКИЙ Р-Н, С. П. ФЕДОСКИНСКОЕ, 

### Считываем файл приоритеты и конкатенируем сверху

In [77]:
prioritets = pd.read_excel('приоритеты/приоритеты.xlsx')

In [78]:
# для прочтения даты в формате timestamp
# prioritets["Дат.зкз.Пок"] = (pd.to_datetime("1899-12-30")+pd.to_timedelta(prioritets["Дат.зкз.Пок"], unit="D"))

# для прочтения даты в формате datetime64





In [79]:
prioritets = prioritets.drop(columns=prioritets.iloc[:, range(3)]) 

In [80]:
prioritets.rename(columns = {'Не исполнено заявки': 'Не исполнено по продукту, тн'}, inplace=True)

In [81]:
prioritets

,№ зкз.Покуп,Дат.зкз.Пок,КССС мат,Номенклатура материала,Заявлено Покупателем,"Не исполнено по продукту, тн",Исполнено опер.отгрузками,имя Покупателя,Вид транспортного средства,СтКМ,Комментарий,ВремяСтКМ,ДатСтКМ,Время,ПланКОНдата,ПланНАЧдата,ИнкТм,Район сбыта,Договор,ДатаСоздан,ДатаИзм,Канал сбыта,Пункт отгрузки,Предписания по таре,Период,Инкот. 2,Регион ГП,Регион Грузополуч.,Маршрут,Пункт разгрузки,Тип дилера,Исполнено факт.отгрузками,Фирменная / не фирменная проду,имя Грузополучателя,КССС Пок,Условие отгрузки,СтКМ продукта
0,5486287,2022-09-05,1599892,"Масло L GENESIS SPEC A5/В5 5W30;б.216,5л",7480.0,0.0,7480.0,"АО ""Авилон АГ""",Авто для фасовки,C,NaN,11:11:29,2022-09-08 00:00:00,08:59:01,2022-09-30,2022-09-07,DDP,Оптовые поставки,20L0408,2022-09-05,2022-09-05,Внутренний рынок,Пермнефтеоргсинтез ООО,крупная фасовка,202209,Москва,77,г. Москва,Автотранспорт(Пермь->Москва),"МОСКОВСКАЯ ОБЛАСТЬ, Г. КОТЕЛЬНИКИ, КОММЕРЧЕСК...",NaN,0.0,Фирм.не аналог имп/з,"АО ""Авилон АГ""",192281,Автотранспорт,A
1,5486138,2022-09-01,227349,"Масло Л АВАНГАРД УЛЬТРА,10W30; б.216,5л",900.0,0.0,900.0,"ООО ""ЛУКОЙЛ-Пермнефтеоргсинтез""",Авто для фасовки,NaN,"Недогруз Август 2022 г., разнярядка № 0005485...",00:00:00,00:00:00,13:45:31,2022-09-30,2022-09-04,DDU,внутрикорп. Транзит,13L0555,2022-09-02,2022-09-06,Внутренний рынок,Пермнефтеоргсинтез ООО,крупная фасовка,202209,г. Тюмень,77,г. Москва,Автотранспорт(Пермь->Тюмень),"П. БОГАНДИНСКИЙ, УЛ. НЕФТЯННИКОВ, 9",NaN,900.0,Фирм.не аналог имп/з,"Филиал ООО ""ЛЛК-Интернешнл"" в Тюмени",198349,Автотранспорт,A
2,5486125,2022-09-02,3435821,Масло Л АВ ПРОФ М5 10W40 б.175кг,3500.0,3500.0,0.0,"ООО ""Виланд""",Авто для фасовки,B,NaN,12:54:05,2022-09-06 00:00:00,12:38:04,2022-09-30,2022-09-04,CPT,Белоруссия,19L0442,2022-09-02,2022-09-06,Экспорт,Пермнефтеоргсинтез ООО,крупная фасовка,202209,г. Минск,0,Нет данных,Автотранспорт(Пермь->г. Минск),"ТЖ:МИНСКАЯ ОБЛ.,ЩОМЫСЛИЦКИЙ С/С,17-Й КМ. А/Д ...",NaN,0.0,NaN,"ООО ""Виланд""",184884,Автотранспорт,B
3,5492131,2022-09-07,189509,"Масло Л ЛЮКС ТУРБО ДИЗЕЛЬ 10W40 б.216,5л",720.0,0.0,720.0,"ООО ""Еврозапчасть""",Авто для фасовки,D,NaN,13:04:31,2022-09-07 00:00:00,11:06:27,2022-09-30,2022-09-09,CPT,Белоруссия,19L0014,2022-09-07,2022-09-07,Экспорт,Пермнефтеоргсинтез ООО,крупная фасовка,202209,г. Минск,7,"г.Минск,Минская обл.",Автотранспорт(Пермь->г. Минск),"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...",NaN,0.0,Фирм. аналог имп/зам,"ООО ""Еврозапчасть""",2111015,Автотранспорт,A
4,5492131,2022-09-07,14913,"Масло Л СУПЕР, 10W40, SG/CD; б.216,5л",5760.0,0.0,5760.0,"ООО ""Еврозапчасть""",Авто для фасовки,D,NaN,13:04:31,2022-09-07 00:00:00,11:06:27,2022-09-30,2022-09-09,CPT,Белоруссия,19L0014,2022-09-07,2022-09-07,Экспорт,Пермнефтеоргсинтез ООО,крупная фасовка,202209,г. Минск,7,"г.Минск,Минская обл.",Автотранспорт(Пермь->г. Минск),"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...",NaN,0.0,Фирм.не аналог имп/з,"ООО ""Еврозапчасть""",2111015,Автотранспорт,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
684,5486887,2022-08-04,3435821,Масло Л АВ ПРОФ М5 10W40 б.175кг,5600.0,5600.0,0.0,"ООО ""М-Траст""",Авто для фасовки,G,"Недогруз Август 2022 г., разнярядка № 0005481909",23:28:16,2022-08-04 00:00:00,10:20:11,2022-09-30,2022-09-07,CPT,Оптовые поставки,20L0848,2022-09-05,2022-09-05,Внутренний рынок,Тюменский Филиал,крупная фасовка,202209,г. Белгород,36,Воронежская обл.,Автотранспорт(Тюмень->Белгород),"Г.БЕЛГОРОД, УЛ.КОНСТАНТИНА ЗАСЛОНОВА, 183",NaN,0.0,NaN,"ООО ""М-Траст""",133086,Автотранспорт,G
685,5490737,2022-08-05,3435821,Масло Л АВ ПРОФ М5 10W40 б.175кг,1400.0,1400.0,0.0,"ООО ""Макон""",Авто для фасовки,A,"Недогруз Август 2022 г., разнярядка № 0005482077",09:48:52,2022-08-08 00:00:00,13:32:53,2022-09-30,2022-09-07,CPT,Оптовые поставки,20L0088,2022-09-05,2022-09-05,Внутренний рынок,Тюменский Филиал,крупная фасовка,202209,г. Тверь,69,Тверская обл.,Автотранспорт(Тюмень->Тверь),

## конкатенируем сверху таблицу приоритетов

In [82]:
orders = pd.concat([prioritets, orders], ignore_index=True)

In [83]:
orders = orders.drop_duplicates(subset=['№ зкз.Покуп','КССС мат'], ignore_index=True)

## Улучшение таблицы orders

In [84]:
orders.isna().sum()

№ зкз.Покуп                          0
Дат.зкз.Пок                          0
КССС мат                             0
Номенклатура материала               0
Заявлено Покупателем                 0
Не исполнено по продукту, тн         0
Исполнено опер.отгрузками            0
имя Покупателя                       0
Вид транспортного средства           0
СтКМ                               650
Комментарий                        883
ВремяСтКМ                            0
ДатСтКМ                            396
Время                                0
ПланКОНдата                          0
ПланНАЧдата                          0
ИнкТм                                0
Район сбыта                          0
Договор                              0
ДатаСоздан                           0
ДатаИзм                              0
Канал сбыта                          0
Пункт отгрузки                       0
Предписания по таре                  0
Период                               0
Инкот. 2                 

In [85]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2164 entries, 0 to 2163
Data columns (total 39 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   № зкз.Покуп                     2164 non-null   int64         
 1   Дат.зкз.Пок                     2164 non-null   datetime64[ns]
 2   КССС мат                        2164 non-null   int64         
 3   Номенклатура материала          2164 non-null   object        
 4   Заявлено Покупателем            2164 non-null   float64       
 5   Не исполнено по продукту, тн    2164 non-null   float64       
 6   Исполнено опер.отгрузками       2164 non-null   float64       
 7   имя Покупателя                  2164 non-null   object        
 8   Вид транспортного средства      2164 non-null   object        
 9   СтКМ                            1514 non-null   object        
 10  Комментарий                     1281 non-null   object        
 11  Врем

In [86]:
orders['Заявлено Покупателем'] = orders['Заявлено Покупателем']/1000

orders['Не исполнено по продукту, тн'] = orders['Не исполнено по продукту, тн']/1000

orders['Исполнено опер.отгрузками'] = orders['Исполнено опер.отгрузками']/1000

In [87]:
def change_name_of_city(city):
    if city == 'Волгоградский НПЗ':
        return 'ВНП'
    elif city == 'Пермнефтеоргсинтез ООО':
        return 'ПНОС'
    else:
        return city

In [88]:
orders['Пункт отгрузки'] = orders.apply(lambda x: change_name_of_city(x['Пункт отгрузки']), axis=1)

## Читаем файлы operfas и группируем (ТОРЖОК)

In [89]:
columns = ['data', 'area', 'number_product', 'balances', 'element']

In [90]:
torzhok = pd.read_excel('operfas_торжок/стоки_торжок.xlsx')

torzhok.drop(columns='Наименование', axis=1, inplace=True)

torzhok.rename(columns={'Пункт отгрузки': 'area', 'Кссс': 'number_product', 'Кг': 'balances'}, inplace=True)

torzhok = torzhok.reindex(columns=['area','number_product',  'balances'])

In [101]:
torzhok

,area,number_product,balances
0,"ООО ""Тебойл Рус"" Торжок",3450871,950.508
1,"ООО ""Тебойл Рус"" Торжок",3450870,922.200
2,"ООО ""Тебойл Рус"" Торжок",3450876,0.000
3,"ООО ""Тебойл Рус"" Торжок",3426103,3261.600
4,"ООО ""Тебойл Рус"" Торжок",3451534,351.120
...,...,...,...
1831,"ООО ""Тебойл Рус"" С.-Петербург",3453943,0.000
1832,"ООО ""Тебойл Рус"" С.-Петербург",1536766,15640.000
1833,"ООО ""Тебойл Рус"" С.-Петербург",3460895,0.000
1834,"ООО ""Тебойл Рус"" С.-Петербург",3461154,0.000


In [102]:
list_operfas = []
for i in os.listdir("operfas"):
    i = pd.read_csv('operfas/'+i, sep='\s+', encoding='CP1251', header = None)
    
    list_operfas.append(i)


for i in list_operfas:
    i.dropna(axis=1, inplace=True)
    i.dropna(axis=0, how='all', inplace=True)
    i.columns = columns
    i['balances'] = i['balances'].apply(lambda x: float(x.split()[0].replace(',', '.')))
    

    
req_3_factory = pd.DataFrame(pd.concat(list_operfas, ignore_index=True))

req_3_factory

,data,area,number_product,balances,element
0,02.04.2022,1078,12722,2220.0,КГ
1,02.04.2022,1078,157569,74900.0,КГ
2,02.04.2022,1078,1620696,9180.0,КГ
3,02.04.2022,1078,3190203,25344.0,КГ
4,03.04.2022,1078,1968,11100.0,КГ
...,...,...,...,...,...
2056,17.04.2022,119388,220687,31680.0,КГ
2057,17.04.2022,119388,1642514,10030.0,КГ
2058,17.04.2022,119388,203951,9720.0,КГ
2059,17.04.2022,119388,3141887,19890.0,КГ


In [92]:
req_3_factory.drop(['data', 'element'], axis=1, inplace=True)

In [93]:
fiz_ostatki = req_3_factory.groupby(['area','number_product'], as_index=False).sum()


In [94]:
fiz_ostatki

,area,number_product,balances
0,1078,1968,31635.000
1,1078,1972,185.000
2,1078,1990,360.000
3,1078,2253,0.000
4,1078,2260,20160.000
...,...,...,...
1496,198349,3400238,527.085
1497,198349,3409231,576.000
1498,198349,3409235,1400.000
1499,198349,3413029,13600.000


### Читаем файл "брак" и удаляем объемы из fiz_ostatki

In [95]:
scrap = pd.read_excel("брак/брак.xlsx",usecols=[0,2,3],names=['number_product','balances', 'area'])

scrap = scrap.reindex(columns=['area','number_product', 'balances'])



In [96]:
scrap

,area,number_product,balances
0,ПНОС,17367,5.632000
1,ПНОС,17437,16.500600
2,ПНОС,1561235,0.925000
3,ПНОС,3051150,0.017900
4,ПНОС,1561243,1.480000
...,...,...,...
152,ПНОС,1670489,0.320896
153,ПНОС,3099062,0.135792
154,ПНОС,3167334,0.430500
155,ПНОС,3167335,3.022194


In [97]:
scrap['balances'] = scrap['balances']*1000
scrap['area'] = 198349

In [98]:
pd.merge(scrap, fiz_ostatki, on=['area','number_product'])

,area,number_product,balances_x,balances_y
0,198349,17367,5632.000,5632.000
1,198349,17437,16500.600,16536.200
2,198349,1561235,925.000,925.000
3,198349,3051150,17.900,53.700
4,198349,1561243,1480.000,1480.000
...,...,...,...,...
146,198349,1670489,320.896,2835.744
147,198349,3099062,135.792,678.960
148,198349,3167334,430.500,430.500
149,198349,3167335,3022.194,3022.350


In [103]:
temp_requirements= pd.merge(fiz_ostatki, scrap, on=['area','number_product'], how='left', suffixes=('_req', '_scrap'))
temp_requirements

,area,number_product,balances_req,balances_scrap
0,1078,1968,31635.000,NaN
1,1078,1972,185.000,NaN
2,1078,1990,360.000,NaN
3,1078,2253,0.000,NaN
4,1078,2260,20160.000,NaN
...,...,...,...,...
1496,198349,3400238,527.085,NaN
1497,198349,3409231,576.000,NaN
1498,198349,3409235,1400.000,NaN
1499,198349,3413029,13600.000,NaN


In [373]:
temp_requirements['balances_scrap'].fillna(0, inplace=True)

In [374]:
temp_requirements

,area,number_product,balances_scrap,balances_req
0,1078,1968,0.0,31635.0
1,1078,1972,0.0,185.0
2,1078,1990,0.0,360.0
3,1078,2253,0.0,0.0
4,1078,2260,0.0,20160.0
...,...,...,...,...
3251,"ООО ""Тебойл Рус"" Торжок",3457308,0.0,1360.0
3252,"ООО ""Тебойл Рус"" Торжок",3460895,0.0,0.0
3253,"ООО ""Тебойл Рус"" Торжок",3461154,0.0,85536.0
3254,"ООО ""Тебойл Рус"" Торжок",3461165,0.0,40374.0


In [375]:
temp_requirements['balances'] = temp_requirements['balances_req']-temp_requirements['balances_scrap']

In [376]:
fiz_ostatki = temp_requirements.drop(['balances_scrap', 'balances_req'], axis=1)

### меняем код города на название

In [377]:
def area_change_name(area):
    if area == 1078:
        return 'Волгоградский НПЗ'
    elif area == 198349:
        return 'Пермнефтеоргсинтез ООО'
    elif area == 119388:
        return 'Тюменский Филиал'
    else:
        return area
        

In [378]:
fiz_ostatki['area'] = fiz_ostatki.apply(lambda x: area_change_name(x['area']), axis=1)

In [379]:
fiz_ostatki

,area,number_product,balances
0,Волгоградский НПЗ,1968,31635.0
1,Волгоградский НПЗ,1972,185.0
2,Волгоградский НПЗ,1990,360.0
3,Волгоградский НПЗ,2253,0.0
4,Волгоградский НПЗ,2260,20160.0
...,...,...,...
3251,"ООО ""Тебойл Рус"" Торжок",3457308,1360.0
3252,"ООО ""Тебойл Рус"" Торжок",3460895,0.0
3253,"ООО ""Тебойл Рус"" Торжок",3461154,85536.0
3254,"ООО ""Тебойл Рус"" Торжок",3461165,40374.0


In [380]:
fiz_ostatki['balances'] = fiz_ostatki['balances']/1000

In [381]:
fiz_ostatki

,area,number_product,balances
0,Волгоградский НПЗ,1968,31.635
1,Волгоградский НПЗ,1972,0.185
2,Волгоградский НПЗ,1990,0.360
3,Волгоградский НПЗ,2253,0.000
4,Волгоградский НПЗ,2260,20.160
...,...,...,...
3251,"ООО ""Тебойл Рус"" Торжок",3457308,1.360
3252,"ООО ""Тебойл Рус"" Торжок",3460895,0.000
3253,"ООО ""Тебойл Рус"" Торжок",3461154,85.536
3254,"ООО ""Тебойл Рус"" Торжок",3461165,40.374


## Расчет физических остатков

In [382]:
def raschet_fiz_ostatki(code_of_product, area, ispolneno):
    if ispolneno!=0:
        if fiz_ostatki[(fiz_ostatki['number_product']==code_of_product) & (fiz_ostatki['area']==area)].empty:
            return 
        if ispolneno > fiz_ostatki.loc[(fiz_ostatki['number_product'] == code_of_product) & (fiz_ostatki['area']==area), 'balances'].iloc[0]:
            fiz_ostatki.loc[(fiz_ostatki['number_product'] == code_of_product) & (fiz_ostatki['area']==area), 'balances'] = 0
            return 
        else: 
            fiz_ostatki.loc[(fiz_ostatki['number_product']==code_of_product) & (fiz_ostatki['area']==area), 'balances'] -= ispolneno
            return 
    

In [383]:
orders.apply(lambda x: raschet_fiz_ostatki(x['КССС мат'], x['Пункт отгрузки'], x['Исполнено опер.отгрузками']), axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
2159    None
2160    None
2161    None
2162    None
2163    None
Length: 2164, dtype: object

## Распределение остатков

In [384]:
def raspredelenie(ksss, neispolneno, area):
    
        
    #if (code_of_product not in req_3_factory_group[req_3_factory_group['area']==area]['number_product'].values) & (area not in req_3_factory_group[req_3_factory_group['number_product']==code_of_product]['area']):
    if fiz_ostatki[(fiz_ostatki['number_product']==ksss) & (fiz_ostatki['area']==area)].empty:
        return 0
       
    elif neispolneno==0:
        return 0
        
    elif neispolneno>0:

        if fiz_ostatki[(fiz_ostatki['number_product']==ksss) & 
                               (fiz_ostatki['area']==area)]['balances'].iloc[0]  >= neispolneno:    
            fiz_ostatki.loc[(fiz_ostatki['number_product']==ksss) & (fiz_ostatki['area']==area), 'balances'] -= neispolneno
            #return neispolneno
            return neispolneno
           
    
        else:
            result = fiz_ostatki[(fiz_ostatki['number_product']==ksss) & 
                               (fiz_ostatki['area']==area)]['balances'].iloc[0]
            fiz_ostatki.loc[(fiz_ostatki['number_product']==ksss) & (fiz_ostatki['area']==area), 'balances'] = 0
            
            return result

In [385]:

orders['Готово к отгрузке по продукту, тн'] = orders.apply(lambda x: raspredelenie(x['КССС мат'], x['Не исполнено по продукту, тн'], x['Пункт отгрузки']), axis=1)
#orders['Готово к отгрузке по продукту, тн'] = orders['Готово к отгрузке по продукту, тн']/1000

In [386]:
orders['Готово к отгрузке по продукту, тн'] = round(orders['Готово к отгрузке по продукту, тн'], 4)

In [387]:
orders

,№ зкз.Покуп,Дат.зкз.Пок,КССС мат,Номенклатура материала,Заявлено Покупателем,"Не исполнено по продукту, тн",Исполнено опер.отгрузками,имя Покупателя,Вид транспортного средства,СтКМ,Комментарий,ВремяСтКМ,ДатСтКМ,Время,ПланКОНдата,ПланНАЧдата,ИнкТм,Район сбыта,Договор,ДатаСоздан,ДатаИзм,Канал сбыта,Пункт отгрузки,Предписания по таре,Период,Инкот. 2,Регион ГП,Регион Грузополуч.,Маршрут,Пункт разгрузки,Тип дилера,Исполнено факт.отгрузками,Фирменная / не фирменная проду,имя Грузополучателя,КССС Пок,Условие отгрузки,СтКМ продукта,Стоимость,Валюта,"Готово к отгрузке по продукту, тн"
0,5486287,2022-09-05,1599892,"Масло L GENESIS SPEC A5/В5 5W30;б.216,5л",7.480,0.000,7.48,"АО ""Авилон АГ""",Авто для фасовки,C,NaN,11:11:29,2022-09-08 00:00:00,08:59:01,2022-09-30 00:00:00,2022-09-07 00:00:00,DDP,Оптовые поставки,20L0408,2022-09-05 00:00:00,2022-09-05 00:00:00,Внутренний рынок,ПНОС,крупная фасовка,202209,Москва,77,г. Москва,Автотранспорт(Пермь->Москва),"МОСКОВСКАЯ ОБЛАСТЬ, Г. КОТЕЛЬНИКИ, КОММЕРЧЕСК...",NaN,0.0,Фирм.не аналог имп/з,"АО ""Авилон АГ""",192281,Автотранспорт,A,NaN,NaN,0.0
1,5486138,2022-09-01,227349,"Масло Л АВАНГАРД УЛЬТРА,10W30; б.216,5л",0.900,0.000,0.90,"ООО ""ЛУКОЙЛ-Пермнефтеоргсинтез""",Авто для фасовки,NaN,"Недогруз Август 2022 г., разнярядка № 0005485...",00:00:00,00:00:00,13:45:31,2022-09-30 00:00:00,2022-09-04 00:00:00,DDU,внутрикорп. Транзит,13L0555,2022-09-02 00:00:00,2022-09-06 00:00:00,Внутренний рынок,ПНОС,крупная фасовка,202209,г. Тюмень,77,г. Москва,Автотранспорт(Пермь->Тюмень),"П. БОГАНДИНСКИЙ, УЛ. НЕФТЯННИКОВ, 9",NaN,900.0,Фирм.не аналог имп/з,"Филиал ООО ""ЛЛК-Интернешнл"" в Тюмени",198349,Автотранспорт,A,NaN,NaN,0.0
2,5486125,2022-09-02,3435821,Масло Л АВ ПРОФ М5 10W40 б.175кг,3.500,3.500,0.00,"ООО ""Виланд""",Авто для фасовки,B,NaN,12:54:05,2022-09-06 00:00:00,12:38:04,2022-09-30 00:00:00,2022-09-04 00:00:00,CPT,Белоруссия,19L0442,2022-09-02 00:00:00,2022-09-06 00:00:00,Экспорт,ПНОС,крупная фасовка,202209,г. Минск,0,Нет данных,Автотранспорт(Пермь->г. Минск),"ТЖ:МИНСКАЯ ОБЛ.,ЩОМЫСЛИЦКИЙ С/С,17-Й КМ. А/Д ...",NaN,0.0,NaN,"ООО ""Виланд""",184884,Автотранспорт,B,NaN,NaN,0.0
3,5492131,2022-09-07,189509,"Масло Л ЛЮКС ТУРБО ДИЗЕЛЬ 10W40 б.216,5л",0.720,0.000,0.72,"ООО ""Еврозапчасть""",Авто для фасовки,D,NaN,13:04:31,2022-09-07 00:00:00,11:06:27,2022-09-30 00:00:00,2022-09-09 00:00:00,CPT,Белоруссия,19L0014,2022-09-07 00:00:00,2022-09-07 00:00:00,Экспорт,ПНОС,крупная фасовка,202209,г. Минск,7,"г.Минск,Минская обл.",Автотранспорт(Пермь->г. Минск),"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...",NaN,0.0,Фирм. аналог имп/зам,"ООО ""Еврозапчасть""",2111015,Автотранспорт,A,NaN,NaN,0.0
4,5492131,2022-09-07,14913,"Масло Л СУПЕР, 10W40, SG/CD; б.216,5л",5.760,0.000,5.76,"ООО ""Еврозапчасть""",Авто для фасовки,D,NaN,13:04:31,2022-09-07 00:00:00,11:06:27,2022-09-30 00:00:00,2022-09-09 00:00:00,CPT,Белоруссия,19L0014,2022-09-07 00:00:00,2022-09-07 00:00:00,Экспорт,ПНОС,крупная фасовка,202209,г. Минск,7,"г.Минск,Минская обл.",Автотранспорт(Пермь->г. Минск),"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...",NaN,0.0,Фирм.не аналог имп/з,"ООО ""Еврозапчасть""",2111015,Автотранспорт,A,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,5485277,2022-08-30,3455335,Масло TERRION ENGINE R 10W40 б.175кг,4.900,4.900,0.00,"АО ""ГК ""ЛогЛаб""",Авто для фасовки,NaN,NaN,00:00:00,NaN,15:30:04,31.08.2022,31.08.2022,DDU,внутрикорп. Транзит,22L0227,30.08.2022,30.08.2022,Внутренний рынок,"ООО ""Тебойл Рус"" Торжок",крупная фасовка,202208,Москва,77,г. Москва,Автотранспорт (Торжок ->Москва),"МО, МЫТИЩИНСКИЙ Р-Н, С. П. ФЕДОСКИНСКОЕ, Д. Е...",NaN,"0,000",NaN,"АО ""ГК ""ЛогЛаб"" Мытищи",3422199,Автотранспорт,A,"1321236,00",RUB,0.0
2160,5485277,2022-08-30,3455339,"Масло TERRION ENGINE R 10W40 к.21,5л",1.632,1.632,0.00,"АО ""ГК ""ЛогЛаб""",Авто для фасовки,NaN,NaN,00:00:00,NaN,15:30:04,31.08.2022,31.08.2022,DDU,внут

## Присвоение даты

In [388]:
graphic = pd.read_excel('график/график.xlsx')

## Добавляем колонку того, сколько осталось произвести

In [389]:
orders['осталось произвести, тн'] = orders['Не исполнено по продукту, тн'] - orders['Готово к отгрузке по продукту, тн']

## Распределение даты

In [390]:
def data_shipment(city, code, required_to_make):
    # проверяем есть ли вообще в списке graphic
    if graphic[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)].empty:
        return '-'
    # если в таблице orders не требуется, то мы ставим черточку
    elif required_to_make == 0:
        return '-'
    # иначе если есть и объема в графике достаточно, то мы    
    elif graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code), 'Объем_фасовки, т'].iloc[0] >= required_to_make:        
        # вычитаем из объема
        # добавить определенное дату, так как будет удалять все значения где код и пункт совпадаем
        data_of_making = graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
                                    (graphic['Дата_производства']==graphic.loc[(graphic['Пункт отгрузки']==city) 
                                    & (graphic['Код КССС']==code), 'Дата_производства'].iloc[0]),'Дата_производства'].iloc[0]
        graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & (graphic['Дата_производства']==data_of_making),
                    'Объем_фасовки, т'] -= required_to_make
        # и возвращаем дату
        data = graphic[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)]['Дата_производства'].dt.date.iloc[0]
        return data
    # иначе если недостаточно объема чем требуется ...
    else:
        # иначе если возвращается только одна дата, возвращает -
        #graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code), 'Объем_фасовки, т'] = 0
        if len(graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)]) == 1:
            return "-"
        # и возращается несколько дат
        elif len(graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)]) > 1:
            # вводим переменную длины, чтобы не выйти за пределы
            len_of_dates = len(graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)])
            # вводим счетчик для того чтобы присвоить правильную дату
            i = 0
            # создаем накопительную переменную 
            storage_var = graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code), 'Объем_фасовки, т'].iloc[0]
            list_of_dates_to_be_deleted = []

            # пока объем фасовки меньше, чем требуется
            while storage_var<required_to_make and len_of_dates>1:
                len_of_dates-=1
                # переходим к следующей
                i+=1
                # и прибавляем к накопительной переменной
                storage_var+= graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code), 'Объем_фасовки, т'].iloc[i]
                list_of_dates_to_be_deleted.append(graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
                                             (graphic['Дата_производства']==graphic.loc[(graphic['Пункт отгрузки']==city) 
                                            & (graphic['Код КССС']==code), 'Дата_производства'].iloc[i-1]),
                                             'Дата_производства'].iloc[0])                    
            #если требуемый объем не накопился то возвращает -
            if storage_var<required_to_make:
                return '-'
            # иначе
            else:
                #нужно обнулить или удалить предыдущие даты, которые схлопнулись
                for j in (list_of_dates_to_be_deleted):
                    graphic.drop(graphic[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
                           (graphic['Дата_производства']==j)].index, inplace=True)
                    i-=1
                data_of_making = graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
                                             (graphic['Дата_производства']==graphic.loc[(graphic['Пункт отгрузки']==city) 
                                            & (graphic['Код КССС']==code), 'Дата_производства'].iloc[i]),
                                             'Дата_производства'].iloc[0]
        
            # вычитаем
            graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
                        (graphic['Дата_производства']==data_of_making), 'Объем_фасовки, т'] = storage_var-required_to_make
 
            data = graphic[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)]['Дата_производства'].dt.date.iloc[i] 
            return data

In [391]:

orders['дата производства'] = orders.apply(lambda x: data_shipment(x['Пункт отгрузки'], x['КССС мат'], x['осталось произвести, тн']), axis=1)

In [392]:
graphic

,Пункт отгрузки,Тип фасовки,Код КССС,Наименование продукции,Дата_производства,"Объем_фасовки, т"
0,ПНОС,"216,5",14913,"Масло Л. СУПЕР 10W40, SG/CD; б.216,5л.",2022-06-12,80.000
1,ПНОС,"216,5",19455,"Масло Л ЛЮКС 10W40, SL/CF; б.216,5л.",2022-06-10,70.000
2,ВНП,"216,5",1779443,"Масло Л марка А; б.216,5л-175кг",2022-06-11,4.000
3,ВНП,"216,5",1779443,"Масло Л марка А; б.216,5л-175кг",2022-06-12,3.000
4,ПНОС,"216,5",3378365,"Масло М-14Г2ЦС; б.216,5л.",2022-06-12,4.000
...,...,...,...,...,...,...
72,ПНОС,1,1659932,"Масло Л. СТАНДАРТ 15W40, SF/CC; нк.1л, РБ.",2022-06-15,1.330
73,ПНОС,60,3173874,Масло L. GENESIS UNIVERSAL DIESEL 5W30; б.48кг.,2022-06-10,1.152
74,ПНОС,60,3257474,Масло Л. АВАНГАРД 15W40 CF-4/SG б.48кг,2022-06-12,1.152
75,Тюменский Филиал,NaN,3045810,"Масло инд-ред. Л-Стило Synth 460; б.216,5л",2022-06-16,1.000


##  Добавление условного столбца

In [393]:
def case_column(zayavleno, ne_ispolneno, ispolneno_oper_otgruzkami, gotovo_k_otgruzke, data_proizvodstva):
    if (zayavleno==0): 
        return "Удалено"
    elif data_proizvodstva != '-':
        return "В графике"
    elif ne_ispolneno == 0:
        return "Отгружено"
    elif ne_ispolneno == gotovo_k_otgruzke:
        return 'Произведено'
    elif (ispolneno_oper_otgruzkami==0) & (gotovo_k_otgruzke==0):
        return 'В обработке'
    else:
        return "Частично произведено"
  
    

In [394]:
orders['Готовность продукта КС'] = orders.apply(lambda x: case_column(x['Заявлено Покупателем'],
            x['Не исполнено по продукту, тн'], x['Исполнено опер.отгрузками'], 
            x['Готово к отгрузке по продукту, тн'], x['дата производства']), axis=1)

In [395]:
orders

,№ зкз.Покуп,Дат.зкз.Пок,КССС мат,Номенклатура материала,Заявлено Покупателем,"Не исполнено по продукту, тн",Исполнено опер.отгрузками,имя Покупателя,Вид транспортного средства,СтКМ,Комментарий,ВремяСтКМ,ДатСтКМ,Время,ПланКОНдата,ПланНАЧдата,ИнкТм,Район сбыта,Договор,ДатаСоздан,ДатаИзм,Канал сбыта,Пункт отгрузки,Предписания по таре,Период,Инкот. 2,Регион ГП,Регион Грузополуч.,Маршрут,Пункт разгрузки,Тип дилера,Исполнено факт.отгрузками,Фирменная / не фирменная проду,имя Грузополучателя,КССС Пок,Условие отгрузки,СтКМ продукта,Стоимость,Валюта,"Готово к отгрузке по продукту, тн","осталось произвести, тн",дата производства,Готовность продукта КС
0,5486287,2022-09-05,1599892,"Масло L GENESIS SPEC A5/В5 5W30;б.216,5л",7.480,0.000,7.48,"АО ""Авилон АГ""",Авто для фасовки,C,NaN,11:11:29,2022-09-08 00:00:00,08:59:01,2022-09-30 00:00:00,2022-09-07 00:00:00,DDP,Оптовые поставки,20L0408,2022-09-05 00:00:00,2022-09-05 00:00:00,Внутренний рынок,ПНОС,крупная фасовка,202209,Москва,77,г. Москва,Автотранспорт(Пермь->Москва),"МОСКОВСКАЯ ОБЛАСТЬ, Г. КОТЕЛЬНИКИ, КОММЕРЧЕСК...",NaN,0.0,Фирм.не аналог имп/з,"АО ""Авилон АГ""",192281,Автотранспорт,A,NaN,NaN,0.0,0.000,-,Отгружено
1,5486138,2022-09-01,227349,"Масло Л АВАНГАРД УЛЬТРА,10W30; б.216,5л",0.900,0.000,0.90,"ООО ""ЛУКОЙЛ-Пермнефтеоргсинтез""",Авто для фасовки,NaN,"Недогруз Август 2022 г., разнярядка № 0005485...",00:00:00,00:00:00,13:45:31,2022-09-30 00:00:00,2022-09-04 00:00:00,DDU,внутрикорп. Транзит,13L0555,2022-09-02 00:00:00,2022-09-06 00:00:00,Внутренний рынок,ПНОС,крупная фасовка,202209,г. Тюмень,77,г. Москва,Автотранспорт(Пермь->Тюмень),"П. БОГАНДИНСКИЙ, УЛ. НЕФТЯННИКОВ, 9",NaN,900.0,Фирм.не аналог имп/з,"Филиал ООО ""ЛЛК-Интернешнл"" в Тюмени",198349,Автотранспорт,A,NaN,NaN,0.0,0.000,-,Отгружено
2,5486125,2022-09-02,3435821,Масло Л АВ ПРОФ М5 10W40 б.175кг,3.500,3.500,0.00,"ООО ""Виланд""",Авто для фасовки,B,NaN,12:54:05,2022-09-06 00:00:00,12:38:04,2022-09-30 00:00:00,2022-09-04 00:00:00,CPT,Белоруссия,19L0442,2022-09-02 00:00:00,2022-09-06 00:00:00,Экспорт,ПНОС,крупная фасовка,202209,г. Минск,0,Нет данных,Автотранспорт(Пермь->г. Минск),"ТЖ:МИНСКАЯ ОБЛ.,ЩОМЫСЛИЦКИЙ С/С,17-Й КМ. А/Д ...",NaN,0.0,NaN,"ООО ""Виланд""",184884,Автотранспорт,B,NaN,NaN,0.0,3.500,-,В обработке
3,5492131,2022-09-07,189509,"Масло Л ЛЮКС ТУРБО ДИЗЕЛЬ 10W40 б.216,5л",0.720,0.000,0.72,"ООО ""Еврозапчасть""",Авто для фасовки,D,NaN,13:04:31,2022-09-07 00:00:00,11:06:27,2022-09-30 00:00:00,2022-09-09 00:00:00,CPT,Белоруссия,19L0014,2022-09-07 00:00:00,2022-09-07 00:00:00,Экспорт,ПНОС,крупная фасовка,202209,г. Минск,7,"г.Минск,Минская обл.",Автотранспорт(Пермь->г. Минск),"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...",NaN,0.0,Фирм. аналог имп/зам,"ООО ""Еврозапчасть""",2111015,Автотранспорт,A,NaN,NaN,0.0,0.000,-,Отгружено
4,5492131,2022-09-07,14913,"Масло Л СУПЕР, 10W40, SG/CD; б.216,5л",5.760,0.000,5.76,"ООО ""Еврозапчасть""",Авто для фасовки,D,NaN,13:04:31,2022-09-07 00:00:00,11:06:27,2022-09-30 00:00:00,2022-09-09 00:00:00,CPT,Белоруссия,19L0014,2022-09-07 00:00:00,2022-09-07 00:00:00,Экспорт,ПНОС,крупная фасовка,202209,г. Минск,7,"г.Минск,Минская обл.",Автотранспорт(Пермь->г. Минск),"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...",NaN,0.0,Фирм.не аналог имп/з,"ООО ""Еврозапчасть""",2111015,Автотранспорт,A,NaN,NaN,0.0,0.000,-,Отгружено
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,5485277,2022-08-30,3455335,Масло TERRION ENGINE R 10W40 б.175кг,4.900,4.900,0.00,"АО ""ГК ""ЛогЛаб""",Авто для фасовки,NaN,NaN,00:00:00,NaN,15:30:04,31.08.2022,31.08.2022,DDU,внутрикорп. Транзит,22L0227,30.08.2022,30.08.2022,Внутренний рынок,"ООО ""Тебойл Рус"" Торжок",крупная фасовка,202208,Москва,77,г. Москва,Автотранспорт (Торжок ->Москва),"МО, МЫТИЩИНСКИЙ Р-Н, С. П. ФЕДОСКИНСКОЕ, Д. Е...",NaN,"0,000",NaN,"АО ""ГК ""ЛогЛаб"" Мытищи",3422199,Автотранспорт,A,"1321236,00",RUB,0.0,4.900,-,В обраб

In [396]:
sum(orders.duplicated())

0

### Просуммировать по разнарядке

в этом случае функция summ_by_order вызывается на объекте, к которой была применена функция и передается в аргументы

In [397]:
 orders.groupby('№ зкз.Покуп', sort=False).sum()

,КССС мат,Заявлено Покупателем,"Не исполнено по продукту, тн",Исполнено опер.отгрузками,Период,Регион ГП,КССС Пок,"Готово к отгрузке по продукту, тн","осталось произвести, тн"
№ зкз.Покуп,,,,,,,,,
5486287,4636159,14.96000,0.000,14.96000,404418,154,384562,0.00,0.000
5486138,227349,0.90000,0.000,0.90000,202209,77,198349,0.00,0.000
5486125,3435821,3.50000,3.500,0.00000,202209,0,184884,0.00,3.500
5492131,11025353,16.09488,0.000,16.09488,1415463,49,14777105,0.00,0.000
5486990,3298747,0.97500,0.975,0.00000,202209,16,818,0.00,0.975
...,...,...,...,...,...,...,...,...,...
5485269,18328,16.28000,16.280,0.00000,202208,30,164832,16.28,0.000
5485274,18328,14.80000,14.800,0.00000,202208,25,164832,14.80,0.000
5485276,20732082,16.17600,16.176,0.00000,1213248,462,20533194,0.00,16.176


In [398]:
def summ_by_order(group_by_column):
 
    summ = group_by_column['Готово к отгрузке по продукту, тн'].sum()
    group_by_column['Готово к отгрузке по разнарядке, тн'] = summ

    return group_by_column




In [399]:
orders = orders.groupby('№ зкз.Покуп',sort=False).apply(summ_by_order)

### Заполним поле "ОБЪЕМ ПО РАЗНАРЯДКЕ"

In [400]:
def summ_by_order_for_value(group):
    summ = group['Не исполнено по продукту, тн'].sum()
    group['ОБЪЕМ ПО РАЗНАРЯДКЕ (Не исполнено по разнарядке всего, тн)'] = summ

    return group

orders = orders.groupby('№ зкз.Покуп',sort=False).apply(summ_by_order_for_value)

In [401]:
orders['% готовности разнарядки'] = orders['Готово к отгрузке по разнарядке, тн']/orders['ОБЪЕМ ПО РАЗНАРЯДКЕ (Не исполнено по разнарядке всего, тн)']
orders['% готовности разнарядки'] = orders['% готовности разнарядки'].round(3)
orders['% готовности разнарядки'].fillna(0.000, inplace=True)

## Признак диллера (справочник)

In [402]:
spravochnik = pd.read_excel('справочник/spravochnik.xlsx', usecols=['Contract_Number'])

In [403]:
spravochnik

,Contract_Number
0,20L0807
1,20L0168
2,20L0004
3,21L0036
4,21L0013
...,...
225,22L0130
226,22L0017
227,21L0047
228,21L1036


In [404]:
orders['Признак дилера'] = orders.apply(lambda x: 1 if x['Договор'] in spravochnik['Contract_Number'].values else 0, axis=1)
#проверяем есть ли номер договора в справочнике, возрвращаем 1 если True, иначе 0

In [405]:
def ready_in_procent(row):
    if row==0:
        return 'Нулевая готовность(0%)'
    elif row<0.7:
        return 'Готовность ниже порога (<70%)'
    elif (row>=0.7) & (row<1):
        return 'Готовность выше порога(>=70% <100%)'
    else:
        return 'Полностью готова к отгрузке(100%)'
orders['Статус готовности по разнарядке'] = orders['% готовности разнарядки'].apply(ready_in_procent)

In [406]:
orders['Дата закрытия разнарядки'] = orders['Дат.зкз.Пок'] + pd.Timedelta(days=60)

In [407]:
orders

,№ зкз.Покуп,Дат.зкз.Пок,КССС мат,Номенклатура материала,Заявлено Покупателем,"Не исполнено по продукту, тн",Исполнено опер.отгрузками,имя Покупателя,Вид транспортного средства,СтКМ,Комментарий,ВремяСтКМ,ДатСтКМ,Время,ПланКОНдата,ПланНАЧдата,ИнкТм,Район сбыта,Договор,ДатаСоздан,ДатаИзм,Канал сбыта,Пункт отгрузки,Предписания по таре,Период,Инкот. 2,Регион ГП,Регион Грузополуч.,Маршрут,Пункт разгрузки,Тип дилера,Исполнено факт.отгрузками,Фирменная / не фирменная проду,имя Грузополучателя,КССС Пок,Условие отгрузки,СтКМ продукта,Стоимость,Валюта,"Готово к отгрузке по продукту, тн","осталось произвести, тн",дата производства,Готовность продукта КС,"Готово к отгрузке по разнарядке, тн","ОБЪЕМ ПО РАЗНАРЯДКЕ (Не исполнено по разнарядке всего, тн)",% готовности разнарядки,Признак дилера,Статус готовности по разнарядке,Дата закрытия разнарядки
0,5486287,2022-09-05,1599892,"Масло L GENESIS SPEC A5/В5 5W30;б.216,5л",7.480,0.000,7.48,"АО ""Авилон АГ""",Авто для фасовки,C,NaN,11:11:29,2022-09-08 00:00:00,08:59:01,2022-09-30 00:00:00,2022-09-07 00:00:00,DDP,Оптовые поставки,20L0408,2022-09-05 00:00:00,2022-09-05 00:00:00,Внутренний рынок,ПНОС,крупная фасовка,202209,Москва,77,г. Москва,Автотранспорт(Пермь->Москва),"МОСКОВСКАЯ ОБЛАСТЬ, Г. КОТЕЛЬНИКИ, КОММЕРЧЕСК...",NaN,0.0,Фирм.не аналог имп/з,"АО ""Авилон АГ""",192281,Автотранспорт,A,NaN,NaN,0.0,0.000,-,Отгружено,0.0,0.000,0.0,0,Нулевая готовность(0%),2022-11-04
1,5486138,2022-09-01,227349,"Масло Л АВАНГАРД УЛЬТРА,10W30; б.216,5л",0.900,0.000,0.90,"ООО ""ЛУКОЙЛ-Пермнефтеоргсинтез""",Авто для фасовки,NaN,"Недогруз Август 2022 г., разнярядка № 0005485...",00:00:00,00:00:00,13:45:31,2022-09-30 00:00:00,2022-09-04 00:00:00,DDU,внутрикорп. Транзит,13L0555,2022-09-02 00:00:00,2022-09-06 00:00:00,Внутренний рынок,ПНОС,крупная фасовка,202209,г. Тюмень,77,г. Москва,Автотранспорт(Пермь->Тюмень),"П. БОГАНДИНСКИЙ, УЛ. НЕФТЯННИКОВ, 9",NaN,900.0,Фирм.не аналог имп/з,"Филиал ООО ""ЛЛК-Интернешнл"" в Тюмени",198349,Автотранспорт,A,NaN,NaN,0.0,0.000,-,Отгружено,0.0,0.000,0.0,0,Нулевая готовность(0%),2022-10-31
2,5486125,2022-09-02,3435821,Масло Л АВ ПРОФ М5 10W40 б.175кг,3.500,3.500,0.00,"ООО ""Виланд""",Авто для фасовки,B,NaN,12:54:05,2022-09-06 00:00:00,12:38:04,2022-09-30 00:00:00,2022-09-04 00:00:00,CPT,Белоруссия,19L0442,2022-09-02 00:00:00,2022-09-06 00:00:00,Экспорт,ПНОС,крупная фасовка,202209,г. Минск,0,Нет данных,Автотранспорт(Пермь->г. Минск),"ТЖ:МИНСКАЯ ОБЛ.,ЩОМЫСЛИЦКИЙ С/С,17-Й КМ. А/Д ...",NaN,0.0,NaN,"ООО ""Виланд""",184884,Автотранспорт,B,NaN,NaN,0.0,3.500,-,В обработке,0.0,3.500,0.0,0,Нулевая готовность(0%),2022-11-01
3,5492131,2022-09-07,189509,"Масло Л ЛЮКС ТУРБО ДИЗЕЛЬ 10W40 б.216,5л",0.720,0.000,0.72,"ООО ""Еврозапчасть""",Авто для фасовки,D,NaN,13:04:31,2022-09-07 00:00:00,11:06:27,2022-09-30 00:00:00,2022-09-09 00:00:00,CPT,Белоруссия,19L0014,2022-09-07 00:00:00,2022-09-07 00:00:00,Экспорт,ПНОС,крупная фасовка,202209,г. Минск,7,"г.Минск,Минская обл.",Автотранспорт(Пермь->г. Минск),"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...",NaN,0.0,Фирм. аналог имп/зам,"ООО ""Еврозапчасть""",2111015,Автотранспорт,A,NaN,NaN,0.0,0.000,-,Отгружено,0.0,0.000,0.0,0,Нулевая готовность(0%),2022-11-06
4,5492131,2022-09-07,14913,"Масло Л СУПЕР, 10W40, SG/CD; б.216,5л",5.760,0.000,5.76,"ООО ""Еврозапчасть""",Авто для фасовки,D,NaN,13:04:31,2022-09-07 00:00:00,11:06:27,2022-09-30 00:00:00,2022-09-09 00:00:00,CPT,Белоруссия,19L0014,2022-09-07 00:00:00,2022-09-07 00:00:00,Экспорт,ПНОС,крупная фасовка,202209,г. Минск,7,"г.Минск,Минская обл.",Автотранспорт(Пермь->г. Минск),"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...",NaN,0.0,Фирм.не аналог имп/з,"ООО ""Еврозапчасть""",2111015,Автотранспорт,A,NaN,NaN,0.0,0.000,-,Отгружено,0.0,0.000,0.0,0,Нулевая готовность(0%),2022-11-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,5485277,2022-08-30,3455335,Масл

## Меняем колонки местами как в форме для рассылки

In [408]:
orders = orders.reindex(columns=['Пункт отгрузки','Дат.зкз.Пок','Дата закрытия разнарядки','№ зкз.Покуп','Договор',
                                 'Канал сбыта', 'ИнкТм', 'Инкот. 2', 'Вид транспортного средства', 'СтКМ', 
                                 'КССС Пок', 'имя Покупателя',
                                 'КССС мат', 'Номенклатура материала', 'Готовность продукта', 'Готовность разнарядки', 'Готовность продукта КС',
                                 'Готовность разнарядки КС', 'Заявлено Покупателем', 'Исполнено опер.отгрузками',
                                 'Не исполнено по продукту, тн', 
                                 'ОБЪЕМ ПО РАЗНАРЯДКЕ (Не исполнено по разнарядке всего, тн)',
                                 'Готово к отгрузке по продукту, тн','Готово к отгрузке по разнарядке, тн',
                                 '% готовности разнарядки', 'Признак от 70%', 'Признак дилера', 
                                 'Признак большой разнарядки', 
                                 'Статус "on hold" в логистическом блоке (до отгрузки фур, загруженных на 100%)',
                                 'Фирм/ГОСТ', 'Пункт разгрузки', 'имя Грузополучателя', 'Комментарий',
                                 'Дата отчета', 'Условие отгрузки', 'Маршрут', 'объем одного поддона тн',
                                 'Не исполнено в поддонах шт', 'Готово к отгрузке поддонов шт',
                                  'ВремяСтКМ', 'ДатСтКМ', 'Время', 'ПланКОНдата', 
                                 'ПланНАЧдата', 'Район сбыта', 'ДатаСоздан', 'ДатаИзм',  'Предписания по таре',
                                 'Период', 'Регион ГП', 'Регион Грузополуч.',   'Тип дилера', 
                                 'Исполнено факт.отгрузками', 'Фирменная / не фирменная проду', 'Стоимость', 
                                 'Валюта', 'осталось произвести, тн', 'дата производства', 'Статус готовности по разнарядке'])

In [409]:
orders

,Пункт отгрузки,Дат.зкз.Пок,Дата закрытия разнарядки,№ зкз.Покуп,Договор,Канал сбыта,ИнкТм,Инкот. 2,Вид транспортного средства,СтКМ,КССС Пок,имя Покупателя,КССС мат,Номенклатура материала,Готовность продукта,Готовность разнарядки,Готовность продукта КС,Готовность разнарядки КС,Заявлено Покупателем,Исполнено опер.отгрузками,"Не исполнено по продукту, тн","ОБЪЕМ ПО РАЗНАРЯДКЕ (Не исполнено по разнарядке всего, тн)","Готово к отгрузке по продукту, тн","Готово к отгрузке по разнарядке, тн",% готовности разнарядки,Признак от 70%,Признак дилера,Признак большой разнарядки,"Статус ""on hold"" в логистическом блоке (до отгрузки фур, загруженных на 100%)",Фирм/ГОСТ,Пункт разгрузки,имя Грузополучателя,Комментарий,Дата отчета,Условие отгрузки,Маршрут,объем одного поддона тн,Не исполнено в поддонах шт,Готово к отгрузке поддонов шт,ВремяСтКМ,ДатСтКМ,Время,ПланКОНдата,ПланНАЧдата,Район сбыта,ДатаСоздан,ДатаИзм,Предписания по таре,Период,Регион ГП,Регион Грузополуч.,Тип дилера,Исполнено факт.отгрузками,Фирменная / не фирменная проду,Стоимость,Валюта,"осталось произвести, тн",дата производства,Статус готовности по разнарядке
0,ПНОС,2022-09-05,2022-11-04,5486287,20L0408,Внутренний рынок,DDP,Москва,Авто для фасовки,C,192281,"АО ""Авилон АГ""",1599892,"Масло L GENESIS SPEC A5/В5 5W30;б.216,5л",NaN,NaN,Отгружено,NaN,7.480,7.48,0.000,0.000,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,"МОСКОВСКАЯ ОБЛАСТЬ, Г. КОТЕЛЬНИКИ, КОММЕРЧЕСК...","АО ""Авилон АГ""",NaN,NaN,Автотранспорт,Автотранспорт(Пермь->Москва),NaN,NaN,NaN,11:11:29,2022-09-08 00:00:00,08:59:01,2022-09-30 00:00:00,2022-09-07 00:00:00,Оптовые поставки,2022-09-05 00:00:00,2022-09-05 00:00:00,крупная фасовка,202209,77,г. Москва,NaN,0.0,Фирм.не аналог имп/з,NaN,NaN,0.000,-,Нулевая готовность(0%)
1,ПНОС,2022-09-01,2022-10-31,5486138,13L0555,Внутренний рынок,DDU,г. Тюмень,Авто для фасовки,NaN,198349,"ООО ""ЛУКОЙЛ-Пермнефтеоргсинтез""",227349,"Масло Л АВАНГАРД УЛЬТРА,10W30; б.216,5л",NaN,NaN,Отгружено,NaN,0.900,0.90,0.000,0.000,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,"П. БОГАНДИНСКИЙ, УЛ. НЕФТЯННИКОВ, 9","Филиал ООО ""ЛЛК-Интернешнл"" в Тюмени","Недогруз Август 2022 г., разнярядка № 0005485...",NaN,Автотранспорт,Автотранспорт(Пермь->Тюмень),NaN,NaN,NaN,00:00:00,00:00:00,13:45:31,2022-09-30 00:00:00,2022-09-04 00:00:00,внутрикорп. Транзит,2022-09-02 00:00:00,2022-09-06 00:00:00,крупная фасовка,202209,77,г. Москва,NaN,900.0,Фирм.не аналог имп/з,NaN,NaN,0.000,-,Нулевая готовность(0%)
2,ПНОС,2022-09-02,2022-11-01,5486125,19L0442,Экспорт,CPT,г. Минск,Авто для фасовки,B,184884,"ООО ""Виланд""",3435821,Масло Л АВ ПРОФ М5 10W40 б.175кг,NaN,NaN,В обработке,NaN,3.500,0.00,3.500,3.500,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,"ТЖ:МИНСКАЯ ОБЛ.,ЩОМЫСЛИЦКИЙ С/С,17-Й КМ. А/Д ...","ООО ""Виланд""",NaN,NaN,Автотранспорт,Автотранспорт(Пермь->г. Минск),NaN,NaN,NaN,12:54:05,2022-09-06 00:00:00,12:38:04,2022-09-30 00:00:00,2022-09-04 00:00:00,Белоруссия,2022-09-02 00:00:00,2022-09-06 00:00:00,крупная фасовка,202209,0,Нет данных,NaN,0.0,NaN,NaN,NaN,3.500,-,Нулевая готовность(0%)
3,ПНОС,2022-09-07,2022-11-06,5492131,19L0014,Экспорт,CPT,г. Минск,Авто для фасовки,D,2111015,"ООО ""Еврозапчасть""",189509,"Масло Л ЛЮКС ТУРБО ДИЗЕЛЬ 10W40 б.216,5л",NaN,NaN,Отгружено,NaN,0.720,0.72,0.000,0.000,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...","ООО ""Еврозапчасть""",NaN,NaN,Автотранспорт,Автотранспорт(Пермь->г. Минск),NaN,NaN,NaN,13:04:31,2022-09-07 00:00:00,11:06:27,2022-09-30 00:00:00,2022-09-09 00:00:00,Белоруссия,2022-09-07 00:00:00,2022-09-07 00:00:00,крупная фасовка,202209,7,"г.Минск,Минская обл.",NaN,0.0,Фирм. аналог имп/зам,NaN,NaN,0.000,-,Нулевая готовность(0%)
4,ПНОС,2022-09-07,2022-11-06,5492131,19L0014,Экспорт,CPT,г. Минск,Авто для фасовки,D,2111015,"ООО ""Еврозапчасть""",14913,"Масло Л СУПЕР, 10W40, SG/CD; б.216,5л",NaN,NaN,Отгружено,NaN,5.760,5.76,0.000,0.000,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...","ООО ""Еврозапчасть""",NaN,NaN,Автотранспорт,Автотранспорт(Пермь->г. Минск),NaN

In [410]:
orders.rename(columns = {'Канал сбыта' : 'Рынок', 'ИнкТм' : 'Инкотермс', 'Инкот. 2' : 'Инкотермс 2', 
                        'имя Покупателя' : 'Покупатель','Заявлено Покупателем' : 'Заявлено Покупателем, тн',
                        'Исполнено опер.отгрузками' : 'Отгружено, тн'}, inplace=True)

### заполним поле "готовность продукта"

In [411]:
from datetime import datetime
current_date = datetime.now().date().strftime('%d-%m-%Y')
print(type(datetime.now().date()))
print(current_date)

<class 'datetime.date'>
13-09-2022


In [412]:
def fill_column_product(readiness_of_the_KS_product, date_of_production):
    if date_of_production != '-':
        return date_of_production
    elif readiness_of_the_KS_product == 'Произведено':
        return current_date
    elif readiness_of_the_KS_product == 'Удалено':
        return '00.01.1900'
    elif readiness_of_the_KS_product == 'Отгружено':
        return '00.01.1900'
    elif readiness_of_the_KS_product == 'Частично произведено':
        return '-'
    elif readiness_of_the_KS_product == 'В обработке':
        return '-'
    

In [413]:
orders['Готовность продукта'] = orders.apply(lambda x: fill_column_product(x['Готовность продукта КС'], x['дата производства']), axis=1)

### заполним поле "Готовность разнарядки КС"

In [414]:
import numpy as np
test_data = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),   columns=['a', 'b', 'c'])

In [415]:
test_data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [416]:
(7 == test_data['a'])

0    False
1    False
2     True
Name: a, dtype: bool

In [417]:
def fill_column_ks(group_by_column):
#     print('BEGIN')
#     print('--------')
#     print(group_by_column['Готовность продукта КС'])
#     print(type(group_by_column['Готовность продукта КС']))
#     print('list: ' + str (group_by_column['Готовность продукта КС'].values))
    
    if ('Удалено' == group_by_column['Готовность продукта КС']).all():
        group_by_column['Готовность разнарядки КС'] = 'Удалено'
        return group_by_column
    if ('Отгружено' == group_by_column['Готовность продукта КС']).all():
        group_by_column['Готовность разнарядки КС'] = 'Отгружено'
        return group_by_column
    if ('В обработке' == group_by_column['Готовность продукта КС']).any():
        group_by_column['Готовность разнарядки КС'] = 'В обработке'
        return group_by_column
    if ('Частично произведено' == group_by_column['Готовность продукта КС']).any():
        group_by_column['Готовность разнарядки КС'] = 'В обработке'
        return group_by_column
    if ('Произведено' == group_by_column['Готовность продукта КС']).any():
        group_by_column['Готовность разнарядки КС'] = 'Произведено'
        return group_by_column
    else:
        group_by_column['Готовность разнарядки КС'] = 'В обработке'
        return group_by_column
       



In [418]:
orders = orders.groupby('№ зкз.Покуп',sort=False).apply(fill_column_ks)

### Заполним поле "Готовность разнарядки"

In [419]:
def fill_column_order(readiness_of_the_KS_product):
    if readiness_of_the_KS_product == 'Произведено':
        return current_date
    elif readiness_of_the_KS_product == 'Удалено':
        return '-'
    elif readiness_of_the_KS_product == 'Отгружено':
        return '00.01.1900'
    elif readiness_of_the_KS_product == 'В обработке':
        return '-'

In [420]:
orders['Готовность разнарядки'] = orders.apply(lambda x: fill_column_order(x['Готовность разнарядки КС']), axis=1)

### Заполним поле "Признак от 70%"

In [421]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2164 entries, 0 to 2163
Data columns (total 59 columns):
 #   Column                                                                         Non-Null Count  Dtype         
---  ------                                                                         --------------  -----         
 0   Пункт отгрузки                                                                 2164 non-null   object        
 1   Дат.зкз.Пок                                                                    2164 non-null   datetime64[ns]
 2   Дата закрытия разнарядки                                                       2164 non-null   datetime64[ns]
 3   № зкз.Покуп                                                                    2164 non-null   int64         
 4   Договор                                                                        2164 non-null   object        
 5   Рынок                                                                          2164

In [422]:
def fill_column_feature(feature):
    if feature >= 69.99:
        return 1
    else:
        return 0

In [423]:
orders['Признак от 70%'] = orders.apply(lambda x: fill_column_feature(x['% готовности разнарядки']), axis=1)

### поменяет формат даты в колонке Дат.зкз.Пок

In [424]:
orders['Дат.зкз.Пок'] = orders['Дат.зкз.Пок'].dt.strftime('%d.%m.%Y')

### добавим процент к колонке "Признак от 70%"

In [425]:
orders['% готовности разнарядки'] = orders['% готовности разнарядки'].astype(str)+"%"

# тест

In [426]:
orders

,Пункт отгрузки,Дат.зкз.Пок,Дата закрытия разнарядки,№ зкз.Покуп,Договор,Рынок,Инкотермс,Инкотермс 2,Вид транспортного средства,СтКМ,КССС Пок,Покупатель,КССС мат,Номенклатура материала,Готовность продукта,Готовность разнарядки,Готовность продукта КС,Готовность разнарядки КС,"Заявлено Покупателем, тн","Отгружено, тн","Не исполнено по продукту, тн","ОБЪЕМ ПО РАЗНАРЯДКЕ (Не исполнено по разнарядке всего, тн)","Готово к отгрузке по продукту, тн","Готово к отгрузке по разнарядке, тн",% готовности разнарядки,Признак от 70%,Признак дилера,Признак большой разнарядки,"Статус ""on hold"" в логистическом блоке (до отгрузки фур, загруженных на 100%)",Фирм/ГОСТ,Пункт разгрузки,имя Грузополучателя,Комментарий,Дата отчета,Условие отгрузки,Маршрут,объем одного поддона тн,Не исполнено в поддонах шт,Готово к отгрузке поддонов шт,ВремяСтКМ,ДатСтКМ,Время,ПланКОНдата,ПланНАЧдата,Район сбыта,ДатаСоздан,ДатаИзм,Предписания по таре,Период,Регион ГП,Регион Грузополуч.,Тип дилера,Исполнено факт.отгрузками,Фирменная / не фирменная проду,Стоимость,Валюта,"осталось произвести, тн",дата производства,Статус готовности по разнарядке
0,ПНОС,05.09.2022,2022-11-04,5486287,20L0408,Внутренний рынок,DDP,Москва,Авто для фасовки,C,192281,"АО ""Авилон АГ""",1599892,"Масло L GENESIS SPEC A5/В5 5W30;б.216,5л",00.01.1900,00.01.1900,Отгружено,Отгружено,7.480,7.48,0.000,0.000,0.0,0.0,0.0%,0,0,NaN,NaN,NaN,"МОСКОВСКАЯ ОБЛАСТЬ, Г. КОТЕЛЬНИКИ, КОММЕРЧЕСК...","АО ""Авилон АГ""",NaN,NaN,Автотранспорт,Автотранспорт(Пермь->Москва),NaN,NaN,NaN,11:11:29,2022-09-08 00:00:00,08:59:01,2022-09-30 00:00:00,2022-09-07 00:00:00,Оптовые поставки,2022-09-05 00:00:00,2022-09-05 00:00:00,крупная фасовка,202209,77,г. Москва,NaN,0.0,Фирм.не аналог имп/з,NaN,NaN,0.000,-,Нулевая готовность(0%)
1,ПНОС,01.09.2022,2022-10-31,5486138,13L0555,Внутренний рынок,DDU,г. Тюмень,Авто для фасовки,NaN,198349,"ООО ""ЛУКОЙЛ-Пермнефтеоргсинтез""",227349,"Масло Л АВАНГАРД УЛЬТРА,10W30; б.216,5л",00.01.1900,00.01.1900,Отгружено,Отгружено,0.900,0.90,0.000,0.000,0.0,0.0,0.0%,0,0,NaN,NaN,NaN,"П. БОГАНДИНСКИЙ, УЛ. НЕФТЯННИКОВ, 9","Филиал ООО ""ЛЛК-Интернешнл"" в Тюмени","Недогруз Август 2022 г., разнярядка № 0005485...",NaN,Автотранспорт,Автотранспорт(Пермь->Тюмень),NaN,NaN,NaN,00:00:00,00:00:00,13:45:31,2022-09-30 00:00:00,2022-09-04 00:00:00,внутрикорп. Транзит,2022-09-02 00:00:00,2022-09-06 00:00:00,крупная фасовка,202209,77,г. Москва,NaN,900.0,Фирм.не аналог имп/з,NaN,NaN,0.000,-,Нулевая готовность(0%)
2,ПНОС,02.09.2022,2022-11-01,5486125,19L0442,Экспорт,CPT,г. Минск,Авто для фасовки,B,184884,"ООО ""Виланд""",3435821,Масло Л АВ ПРОФ М5 10W40 б.175кг,-,-,В обработке,В обработке,3.500,0.00,3.500,3.500,0.0,0.0,0.0%,0,0,NaN,NaN,NaN,"ТЖ:МИНСКАЯ ОБЛ.,ЩОМЫСЛИЦКИЙ С/С,17-Й КМ. А/Д ...","ООО ""Виланд""",NaN,NaN,Автотранспорт,Автотранспорт(Пермь->г. Минск),NaN,NaN,NaN,12:54:05,2022-09-06 00:00:00,12:38:04,2022-09-30 00:00:00,2022-09-04 00:00:00,Белоруссия,2022-09-02 00:00:00,2022-09-06 00:00:00,крупная фасовка,202209,0,Нет данных,NaN,0.0,NaN,NaN,NaN,3.500,-,Нулевая готовность(0%)
3,ПНОС,07.09.2022,2022-11-06,5492131,19L0014,Экспорт,CPT,г. Минск,Авто для фасовки,D,2111015,"ООО ""Еврозапчасть""",189509,"Масло Л ЛЮКС ТУРБО ДИЗЕЛЬ 10W40 б.216,5л",00.01.1900,00.01.1900,Отгружено,Отгружено,0.720,0.72,0.000,0.000,0.0,0.0,0.0%,0,0,NaN,NaN,NaN,"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...","ООО ""Еврозапчасть""",NaN,NaN,Автотранспорт,Автотранспорт(Пермь->г. Минск),NaN,NaN,NaN,13:04:31,2022-09-07 00:00:00,11:06:27,2022-09-30 00:00:00,2022-09-09 00:00:00,Белоруссия,2022-09-07 00:00:00,2022-09-07 00:00:00,крупная фасовка,202209,7,"г.Минск,Минская обл.",NaN,0.0,Фирм. аналог имп/зам,NaN,NaN,0.000,-,Нулевая готовность(0%)
4,ПНОС,07.09.2022,2022-11-06,5492131,19L0014,Экспорт,CPT,г. Минск,Авто для фасовки,D,2111015,"ООО ""Еврозапчасть""",14913,"Масло Л СУПЕР, 10W40, SG/CD; б.216,5л",00.01.1900,00.01.1900,Отгружено,Отгружено,5.760,5.76,0.000,0.000,0.0,0.0,0.0%,0,0,NaN,NaN,NaN,"СВХ ТЛЦ ""МИНСК-БЕЛТАМОЖСЕРВИС-2"", МИНСКИЙ Р-Н...","ООО ""

In [427]:
len(orders.columns.unique())

59

### сохраним файл

In [428]:
orders.to_excel("orders_outputs_6.xlsx",sheet_name='Sheet_name_1')

# Конец файла